# DAG full_WB

In [1]:
import re
from typing import Any, Dict, List
import pymysql
# from airflow import DAG
# from airflow.operators.python import PythonOperator
import os 
import datetime
import requests
import time
from sqlalchemy import create_engine
import numpy as np
import openpyxl
import gspread
import gspread_dataframe as gd
from tqdm import tqdm
from sys import exit
import pandas as pd
import sys
import glob
link_to_table = 'https://docs.google.com/spreadsheets/d/1A1TyZPeq9r2HiDi-CHfmTfRpuAQYToib1vqyTx4ZyRE/edit#gid=0'
df = pd.read_excel('/'.join(link_to_table.split('/')[:-1])+'/export?format=xlsx')
PATH, df = [dict(df.loc[x]) for x in list(df.index)], None
i = 0
while len([py for py in glob.glob(f"{PATH[i]['path_downloads']}*.xlsx")]) == 0:
    i +=1
path_downloads = PATH[i]['path_downloads']
path_algoritm = PATH[i]['path_algoritm']
sys.path.append(f'{path_algoritm}Модули')
import constants_for_marketplace_metrics_dags as const
import working_with_marketplace_metrics_dags as wwm
import maslow_working_module as mwm

bd_param = const.bd_param
engine_param = const.engine_param
maslow_json = path_downloads + const.maslow_json
gc = gspread.service_account(filename=maslow_json)
args = const.args
os.environ['NO_PROXY'] = 'URL'

def wb_orders_metrics(path_wb_orders_metrics):
    df = mwm.get_tabl_from_bd('wb_orders2', date_column='date', schema=mwm.ip_schema)[['date', 'supplierarticle']]\
    .rename(columns={'supplierarticle': 'Артикул'})
    df['Артикул'] = df['Артикул'].astype(str)
    df['Артикул'] = [x.replace('/', '') for x in list(df['Артикул'])]
    df = wwm.clear_article(df, 'Артикул')
    df['date'] = [pd.to_datetime(x).strftime('%d-%B-%Y') for x in list(df['date'])]
    df_0 = pd.DataFrame(list(set(df['Артикул'])), columns=['Артикул'])
    for row in [y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in list(df['date'].unique())])]:
        df_0 = df_0.merge(df[df['date']==row].groupby('Артикул')['date'].count()\
        .reset_index(drop=False).rename(columns={'date': row}), on='Артикул', how='left')
    df_0 = df_0.fillna(0)
    df_0.iloc[:, 1:] = df_0.iloc[:, 1:].astype(int)
    df, df_0 = df_0, None
    wwm.unload_metrics(path_metrics=path_wb_orders_metrics, 
                       data1=df, 
                       column='Артикул')
    T_wb_orders_metrics = wwm.get_T_metrics(path_metrics=path_wb_orders_metrics)
    sh = gc.open("full_WB")
    list_name = 'wb_orders_metrics'
    i = [x for x in list(range(len(sh.worksheets()))) if list_name in str(sh.worksheets()[x])][0]
    ws = sh.get_worksheet(i)
    ws.clear()
    gd.set_with_dataframe(worksheet=ws,
                          dataframe=T_wb_orders_metrics, 
                          include_index=True,
                          include_column_header=True,
                          resize=True)

def wb_fbo_stocks_metrics(path_wb_fbo_stocks_metrics):
    df = mwm.get_tabl_from_bd(table_name='wb_fbo_stocks', date_column='date')
    df = df[['supplierarticle', 'quantityfull', 'date']]
    df = df.rename(columns={'supplierarticle': 'Артикул'})
    df['Артикул'] = df['Артикул'].astype(str)
    df['Артикул'] = [x.replace('/', '') for x in list(df['Артикул'])]
    df = wwm.clear_article(df, 'Артикул')
    df['date'] = [pd.to_datetime(x).strftime('%d-%B-%Y') for x in list(df['date'])]
    df['quantityfull'] = df['quantityfull'].astype(int)
    df_0 = pd.DataFrame(list(set(df['Артикул'])), columns=['Артикул'])
    for row in [y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in list(df['date'].unique())])]:
        df_0 = df_0.merge(df[df['date']==row].groupby('Артикул')['quantityfull'].sum()\
        .reset_index(drop=False).rename(columns={'quantityfull': row}), on='Артикул', how='left')
    df_0 = df_0.fillna(0)
    df_0.iloc[:, 1:] = df_0.iloc[:, 1:].astype(int)
    df, df_0 = df_0, None
    wwm.unload_metrics(path_metrics=path_wb_fbo_stocks_metrics, 
                       data1=df, 
                       column='Артикул')
    T_wb_fbo_stocks_metrics = wwm.get_T_metrics(path_metrics=path_wb_fbo_stocks_metrics)
    sh = gc.open("full_WB")
    list_name = 'wb_fbo_stocks_metrics'
    i = [x for x in list(range(len(sh.worksheets()))) if list_name in str(sh.worksheets()[x])][0]
    ws = sh.get_worksheet(i)
    ws.clear()
    gd.set_with_dataframe(worksheet=ws,
                          dataframe=T_wb_fbo_stocks_metrics, 
                          include_index=True,
                          include_column_header=True,
                          resize=True)

def wb_fbs_stocks_metrics(path_wb_fbs_stocks_metrics):
    wb_fbs_stocks_metrics = mwm.get_tabl_from_bd('wb_fbs_stocks', date_column='date')
    wb_fbs_stocks_metrics['Баркод'] = wb_fbs_stocks_metrics['Баркод'].astype(int)
    wb_list_of_nomenclatures = mwm.get_tabl_from_bd('wb_list_of_nomenclatures', flag=1)
    wb_list_of_nomenclatures['Баркод'] = wb_list_of_nomenclatures['Баркод'].astype(int)
    wb_fbs_stocks_metrics, wb_list_of_nomenclatures = wb_fbs_stocks_metrics\
                                                      .merge(wb_list_of_nomenclatures[['Артикул_продавца', 'Баркод']], 
                                                             on='Баркод', 
                                                             how='left')\
                                                      .rename(columns={'Артикул_продавца': 'Артикул'})\
                                                      .drop(['Баркод', 'Склад'], axis=1)\
                                                      , None
    wb_fbs_stocks_metrics['Артикул'] = wb_fbs_stocks_metrics['Артикул'].astype(str)
    df, wb_fbs_stocks_metrics = wb_fbs_stocks_metrics, None
    df['Остатки_fbs'] = df['Остатки_fbs'].astype(int)
    df['date'] = [pd.to_datetime(x).strftime('%d-%B-%Y') for x in list(df['date'])]
    df_0 = pd.DataFrame(list(set(df['Артикул'])), columns=['Артикул'])
    for row in [y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in list(df['date'].unique())])]:
        df_0 = df_0.merge(df[df['date']==row].groupby('Артикул')['Остатки_fbs'].sum()\
        .reset_index(drop=False).rename(columns={'Остатки_fbs': row}), on='Артикул', how='left')
    df_0 = df_0.fillna(0)
    df, df_0 = df_0, None
    df = df.fillna(0)
    df.iloc[:, 1:] = df.iloc[:, 1:].astype(int)
    wwm.unload_metrics(path_metrics=path_wb_fbs_stocks_metrics, 
                       data1=df, 
                       column='Артикул')
    T_wb_fbs_stocks_metrics = wwm.get_T_metrics(path_metrics=path_wb_fbs_stocks_metrics)
    sh = gc.open("full_WB")
    list_name = 'wb_fbs_stocks_metrics'
    i = [x for x in list(range(len(sh.worksheets()))) if list_name in str(sh.worksheets()[x])][0]
    ws = sh.get_worksheet(i)
    ws.clear()
    gd.set_with_dataframe(worksheet=ws,
                          dataframe=T_wb_fbs_stocks_metrics, 
                          include_index=True,
                          include_column_header=True,
                          resize=True)
    
def wb_price_metrics(path_wb_price_metrics):
    df = mwm.get_tabl_from_bd('wb_list_of_nomenclatures', flag=1)[['Артикул_wb', 
                                                                   'Артикул_продавца']]\
    .rename(columns={'Артикул_продавца': 'Артикул', 'Артикул_wb': 'nmid'})\
    .merge(mwm.get_tabl_from_bd('wb_prices', date_column='date'), on='nmid', how='inner')[['Артикул', 
                                                                                       'price', 
                                                                                       'discount', 
                                                                                       'date']]
    df['Артикул'] = df['Артикул'].astype(str)
    df['Цена'] = df['price'] * ((100 - df['discount'])/100)
    df = df[['Артикул', 'Цена', 'date']]
    df['date'] = [pd.to_datetime(x).strftime('%d-%B-%Y') for x in list(df['date'])]
    df_0 = pd.DataFrame(list(set(df['Артикул'])), columns=['Артикул'])
    for row in [y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in list(df['date'].unique())])]:
        df_0 = df_0.merge(df[df['date']==row].groupby('Артикул')['Цена'].mean()\
        .reset_index(drop=False).rename(columns={'Цена': row}), on='Артикул', how='left')
    df_0 = df_0.fillna(0)
    df, df_0 = df_0, None
    wwm.unload_metrics(path_metrics=path_wb_price_metrics, 
                       data1=df, 
                       column='Артикул')
    T_wb_price_metrics = wwm.get_T_metrics(path_metrics=path_wb_price_metrics)
    sh = gc.open("full_WB")
    list_name = 'wb_price_metrics'
    i = [x for x in list(range(len(sh.worksheets()))) if list_name in str(sh.worksheets()[x])][0]
    ws = sh.get_worksheet(i)
    ws.clear()
    gd.set_with_dataframe(worksheet=ws,
                          dataframe=T_wb_price_metrics, 
                          include_index=True,
                          include_column_header=True,
                          resize=True)            
    
# with DAG(
#     dag_id='full_WB_dag',
#     default_args=args,
#     schedule_interval='40 05 * * *',
#     start_date=datetime.datetime(2023, 11, 13),
#     catchup=False,
#     dagrun_timeout=datetime.timedelta(minutes=10)
# ) as dag:

#     wb_orders_metrics_task = PythonOperator(task_id = 'wb_orders_metrics',
#                                             python_callable=wb_orders_metrics, 
#                                             op_kwargs={'path_wb_orders_metrics': wwm.path_wb_orders_metrics},
#                                             dag=dag)
    
#     wb_fbo_stocks_metrics_task = PythonOperator(task_id = 'wb_fbo_stocks_metrics',
#                                           python_callable=wb_fbo_stocks_metrics, 
#                                           op_kwargs={'path_wb_fbo_stocks_metrics': wwm.path_wb_fbo_stocks_metrics},
#                                           dag=dag)
    
#     wb_fbs_stocks_metrics_task = PythonOperator(task_id = 'wb_fbs_stocks_metrics',
#                                           python_callable=wb_fbs_stocks_metrics, 
#                                           op_kwargs={'path_wb_fbs_stocks_metrics': wwm.path_wb_fbs_stocks_metrics},
#                                           dag=dag)
    
#     wb_price_metrics_task = PythonOperator(task_id = 'wb_price_metrics',
#                                            python_callable=wb_price_metrics, 
#                                            op_kwargs={'path_wb_price_metrics': wwm.path_wb_price_metrics},
#                                            dag=dag)
    
#     wb_orders_metrics_task >> \
#     wb_fbo_stocks_metrics_task >> \
#     wb_fbs_stocks_metrics_task >> \
#     wb_price_metrics_task

In [2]:
wb_orders_metrics(wwm.path_wb_orders_metrics)

Были изменения, датафрейм выгружен


In [3]:
wb_fbo_stocks_metrics(wwm.path_wb_fbo_stocks_metrics)

Были изменения, датафрейм выгружен


In [4]:
wb_fbs_stocks_metrics(wwm.path_wb_fbs_stocks_metrics)

Были изменения, датафрейм выгружен


In [5]:
wb_price_metrics(wwm.path_wb_price_metrics)

Были изменения, датафрейм выгружен


In [7]:
df = mwm.get_tabl_from_bd(table_name='wb_fbo_stocks', date_column='date')
df = df[['supplierarticle', 'quantityfull', 'date']]
df = df.rename(columns={'supplierarticle': 'Артикул'})
df['Артикул'] = df['Артикул'].astype(str)
df['Артикул'] = [x.replace('/', '') for x in list(df['Артикул'])]
df = wwm.clear_article(df, 'Артикул')
df['date'] = [pd.to_datetime(x).strftime('%d-%B-%Y') for x in list(df['date'])]
df['quantityfull'] = df['quantityfull'].astype(int)
df_0 = pd.DataFrame(list(set(df['Артикул'])), columns=['Артикул'])
for row in [y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in list(df['date'].unique())])]:
    df_0 = df_0.merge(df[df['date']==row].groupby('Артикул')['quantityfull'].sum()\
    .reset_index(drop=False).rename(columns={'quantityfull': row}), on='Артикул', how='left')
df_0 = df_0.fillna(0)
df_0.iloc[:, 1:] = df_0.iloc[:, 1:].astype(int)
df, df_0 = df_0, None

In [9]:
df = mwm.get_tabl_from_bd(table_name='wb_fbo_stocks', date_column='date')

In [10]:
df

,lastchangedate,warehousename,supplierarticle,nmid,barcode,quantity,inwaytoclient,inwayfromclient,quantityfull,category,subject,brand,techsize,price,discount,issupply,isrealization,sccode,date
0,2024-02-20T03:31:02,Электросталь,22800,10019906,7432141550767,191,0,0,191,Аксессуары,Чехлы для ноутбуков,SS&Y Group,0,8200,85,true,false,Tech,2024-02-21
1,2024-02-20T03:32:44,Казань,50390,37363897,7432141557156,556,0,0,556,Красота,Пинцеты,Revolut,0,1571,86,true,false,Tech,2024-02-21
2,2024-02-20T03:32:44,Тула,Z0111,48157059,7432141559163,208,0,0,208,Посуда и инвентарь,Кружки,Limpid Side,0,3381,79,true,false,Tech,2024-02-21
3,2024-02-20T03:32:44,Казань,G0194,69162543,7432141556817,0,0,0,0,Хозяйственные товары,Мусорные ведра,Birdhouse,0,8697,67,true,false,Tech,2024-02-21
4,2024-02-20T03:32:44,Екатеринбург - Перспективный 12,A3505,77673950,7432141556090,0,0,0,0,Хранение вещей,Органайзеры для хранения,Parchment,0,1849,47,true,false,Tech,2024-02-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70273,2024-02-03T15:20:05,Белые Столбы,19300,8424489,7432141550517,1,0,0,1,Красота,Инструменты для чистки лица,Revolut,0,15364,89,false,true,0900-0876,2024-02-20
70274,2024-02-03T16:50:04,Тула,G0050,33291406,7432141556663,99,0,0,99,Аксессуары для волос,Ободки,JewelryMeverly,0,3200,95,true,false,Tech,2024-02-20
70275,2024-02-03T16:52:04,Белые Столбы,29100,11368103,7432141550889,0,0,1,1,Красота,Щетки косметические,Revolut,0,1075,84,false,true,0900-0876,2024-02-20
70276,2024-02-03T17:24:05,Невинномысск,A1603,151123319,4607880295029,1,0,0,1,Спортивный товар,Эспандеры,Shark Fit,0,1277,35,true,false,Tech,2024-02-20


In [16]:
P = wwm.path_wb_orders_metrics

In [17]:
df = mwm.select_table('dds_wb_orders', schema=mwm.dds_schema)

In [18]:
df = df[df['account']=='ir'].reset_index(drop=True)

In [19]:
df = df.drop('account', axis=1)

In [20]:
df

,barcode,date,orders
0,7432141050058,2020-02-23,17
1,7432141050065,2020-02-23,11
2,7432141550507,2020-02-23,3
3,7432141550517,2020-02-23,4
4,7432141550521,2020-02-23,2
...,...,...,...
438377,7432141950713,2024-02-21,7
438378,7432141950746,2024-02-21,3
438379,7432141950750,2024-02-21,2
438380,7432141950753,2024-02-21,2


In [21]:
ab = mwm.select_table('article_barcodes')

In [23]:
df = df.merge(ab, on='barcode', how='outer')

In [25]:
df = df[['article_name', 'date', 'orders']]

In [28]:
df = df.dropna().reset_index(drop=True)

In [30]:
df['orders'] = df['orders'].astype(int)

In [32]:
df['article_name'] = df['article_name'].astype(str)

In [33]:
df = wwm.expand_df(df,                    
                   key_column='article_name',
                   result_column='orders',
                   func='sum')

In [35]:
df = df.rename(columns={'article_name': 'Артикул'})

In [36]:
mwm.unique_items(df)

In [38]:
df.to_excel(wwm.path_wb_orders_metrics, index=True)

In [60]:
wb_fbo_stocks_metrics(wwm.path_wb_orders_metrics)

Exception ignored in: <function ZipFile.__del__ at 0x7fa1b8b72320>
Traceback (most recent call last):
  File "/Users/macbook/anaconda3/envs/work_venv/lib/python3.10/zipfile.py", line 1821, in __del__
    self.close()
  File "/Users/macbook/anaconda3/envs/work_venv/lib/python3.10/zipfile.py", line 1838, in close
    self.fp.seek(self.start_dir)
TimeoutError: [Errno 60] Operation timed out


Были изменения, датафрейм выгружен


In [61]:
wb_fbo_stocks_metrics(wwm.path_wb_orders_metrics)

In [62]:
wb_price_metrics(wwm.path_wb_orders_metrics)

Exception ignored in: <function ZipFile.__del__ at 0x7fa1b8b72320>
Traceback (most recent call last):
  File "/Users/macbook/anaconda3/envs/work_venv/lib/python3.10/zipfile.py", line 1821, in __del__
    self.close()
  File "/Users/macbook/anaconda3/envs/work_venv/lib/python3.10/zipfile.py", line 1838, in close
    self.fp.seek(self.start_dir)
TimeoutError: [Errno 60] Operation timed out


Были изменения, датафрейм выгружен


In [57]:
df.sort_values(by='Артикул').reset_index(drop=True).to_excel(wwm.path_wb_orders_metrics, index=False)

In [58]:
wwm.path_wb_orders_metrics

'/Users/macbook/Library/CloudStorage/GoogleDrive-maslowdmitry92@gmail.com/Мой диск/Алгоритмы Маслов/FULL_DF/wb_orders_metrics.xlsx'

In [46]:
T_wb_fbs_stocks_metrics = wwm.get_T_metrics(path_metrics=wwm.path_wb_orders_metrics)

ParserError: Unknown string format: Артикул

In [ ]:
T_wb_fbs_stocks_metrics

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1212 entries, 0 to 1211
Columns: 1461 entries, Артикул to 21-February-2024
dtypes: int64(1460), object(1)
memory usage: 13.5+ MB


In [50]:
df = df.sort_values(by='Артикул').reset_index(drop=True)

In [53]:
T_metrics = df.copy()

In [54]:
    T_metrics.columns = ['Артикул']+[pd.to_datetime(x) for x in list(T_metrics.columns)[1:]]
    T_metrics = T_metrics[['Артикул']+sorted(T_metrics.columns[1:])[::-1]]
    T_metrics = T_metrics.T
    T_metrics = T_metrics.reset_index(drop=False)
    T_metrics.columns = list(T_metrics.loc[0])
    T_metrics = T_metrics.iloc[1:, :].reset_index(drop=True)
    T_metrics.index = T_metrics['Артикул']
    T_metrics = T_metrics.drop('Артикул', axis=1)

In [55]:
T_metrics

,16000,16801,16802,16803,16804,16920,16930,16931,16932,16950,...,Z0242,Z0243,Z0244,Z0245,Z0246,Z0247,Z0400,Z0401,Z0500,Z0501
Артикул,,,,,,,,,,,,,,,,,,,,,
2024-02-21,0,0,0,0,0,0,0,3,5,2,...,0,0,0,0,6,5,6,7,5,2
2024-02-20,0,0,0,2,0,0,1,1,2,2,...,0,0,2,1,7,2,4,6,5,4
2024-02-19,0,0,1,0,0,0,2,5,3,2,...,0,0,3,0,3,7,7,11,7,2
2024-02-18,0,0,0,0,0,1,1,11,3,0,...,0,0,2,1,11,9,4,7,8,2
2024-02-17,0,0,1,0,0,0,0,4,3,7,...,0,0,4,0,12,8,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-27,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-02-26,0,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-02-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
path_wb_orders_metrics = wwm.path_wb_orders_metrics

df = mwm.get_tabl_from_bd('wb_orders2', date_column='date', schema=mwm.ip_schema)[['date', 'supplierarticle']]\
.rename(columns={'supplierarticle': 'Артикул'})
df['Артикул'] = df['Артикул'].astype(str)
df['Артикул'] = [x.replace('/', '') for x in list(df['Артикул'])]
df = wwm.clear_article(df, 'Артикул')
df['date'] = [pd.to_datetime(x).strftime('%d-%B-%Y') for x in list(df['date'])]
df_0 = pd.DataFrame(list(set(df['Артикул'])), columns=['Артикул'])
for row in [y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in list(df['date'].unique())])]:
    df_0 = df_0.merge(df[df['date']==row].groupby('Артикул')['date'].count()\
    .reset_index(drop=False).rename(columns={'date': row}), on='Артикул', how='left')
df_0 = df_0.fillna(0)
df_0.iloc[:, 1:] = df_0.iloc[:, 1:].astype(int)
df, df_0 = df_0, None
wwm.unload_metrics(path_metrics=path_wb_orders_metrics, 
                   data1=df, 
                   column='Артикул')
T_wb_orders_metrics = wwm.get_T_metrics(path_metrics=path_wb_orders_metrics)
# sh = gc.open("full_WB")
# list_name = 'wb_orders_metrics'
# i = [x for x in list(range(len(sh.worksheets()))) if list_name in str(sh.worksheets()[x])][0]
# ws = sh.get_worksheet(i)
# ws.clear()
# gd.set_with_dataframe(worksheet=ws,
#                       dataframe=T_wb_orders_metrics, 
#                       include_index=True,
#                       include_column_header=True,
#                       resize=True)

NameError: name 'path_wb_orders_metrics' is not defined

In [3]:
df = mwm.get_tabl_from_bd('wb_orders2', date_column='date', schema=mwm.ip_schema)[['date', 'supplierarticle']]\
.rename(columns={'supplierarticle': 'Артикул'})
df['Артикул'] = df['Артикул'].astype(str)
df['Артикул'] = [x.replace('/', '') for x in list(df['Артикул'])]
df = wwm.clear_article(df, 'Артикул')
df['date'] = [pd.to_datetime(x).strftime('%d-%B-%Y') for x in list(df['date'])]
df_0 = pd.DataFrame(list(set(df['Артикул'])), columns=['Артикул'])
for row in [y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in list(df['date'].unique())])]:
    df_0 = df_0.merge(df[df['date']==row].groupby('Артикул')['date'].count()\
    .reset_index(drop=False).rename(columns={'date': row}), on='Артикул', how='left')
df_0 = df_0.fillna(0)
df_0.iloc[:, 1:] = df_0.iloc[:, 1:].astype(int)
df, df_0 = df_0, None

In [4]:
df

,Артикул,11-February-2024,12-February-2024,13-February-2024,14-February-2024,15-February-2024,16-February-2024,17-February-2024,18-February-2024,19-February-2024
0,A8402,10,9,8,9,8,6,2,13,11
1,B0131,1,3,9,5,6,8,5,5,2
2,19606,2,0,4,2,5,3,7,6,1
3,18311,0,0,1,0,1,1,0,2,2
4,H0071,0,0,0,5,1,4,5,2,3
...,...,...,...,...,...,...,...,...,...,...
887,19811,3,7,3,3,2,6,5,2,6
888,50533,12,6,15,7,8,14,1,0,1
889,G0012,1,6,1,2,1,2,0,3,1
890,J0030,0,0,1,3,2,5,3,2,2


In [ ]:
df